In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [17]:
df_history = pd.read_csv(x
                                    "Thalia_history_ersteBereinigung.csv")

In [18]:
df_relevante_iids = pd.read_csv(x
                                    "Thalia_transactions_zweiteBereinigung_iids.csv")

##### Relevante Kunden und Hist mergen

In [21]:
df_h = pd.merge(df_history, df_relevante_iids, on = ["iid"])
df_h[["iid","attrtype", "text", "month_count"]].head(5)

,iid,attrtype,text,month_count
0,100055559,HY,|msg=NL_Mai_1|subject=St%C3%BCck+der+Stunde%3A...,0
1,100055559,NO,|msg=NL_Mai_1|userAgent=Apple Mail (OS X / Per...,0
2,100055559,HY,|msg=NL_Mai_2|subject=Online+Premiere+%22Kabal...,0
3,100055559,NO,|msg=NL_Mai_2|userAgent=Apple Mail (OS X / Per...,0
4,100055559,HY,|msg=NL_Corona_1|subject=Sondernewsletter%3A+J...,1


##### Erstellung des "df_neueSpalte" für die Betrachtung sämtlicher Infos zwischen den "|"

In [24]:
neueSpalten = [ line.strip().split('|') for line in df_h["text"] ]

##### Ein neues DataFrame df_user_agent und eine neue Spalte "User agent", die noch sämtliche Informationen danach enthält

In [25]:
df_user_agent = pd.DataFrame()
df_user_agent["User Agent"] = df_h["text"].str.extract(r'userAgent=(.*)', expand=False)
df_user_agent["User Agent"] = df_user_agent["User Agent"].apply(lambda zeile: zeile.split("|")[0] if isinstance(zeile, str) else zeile)

In [26]:
df_user_agent.head()

,User Agent
0,NaN
1,Apple Mail (OS X / Personal computer)
2,NaN
3,Apple Mail (OS X / Personal computer)
4,NaN


##### Ermittlun aller iids die mindestens einmal mit Apple Mail/Gmail oder Proxy-Server eine Email geöffnet haben

In [27]:
df_user_agent["User Agent"].fillna("unknown", inplace=True)
df_h["User Agent"] = df_user_agent["User Agent"]
df_h.head()

,iid,attrtype,akey,date,text,month_count,User Agent
0,100055559,HY,107601295,2020-05-07 11:53:17,|msg=NL_Mai_1|subject=St%C3%BCck+der+Stunde%3A...,0,unknown
1,100055559,NO,107601295,2020-05-08 09:50:15,|msg=NL_Mai_1|userAgent=Apple Mail (OS X / Per...,0,Apple Mail (OS X / Personal computer)
2,100055559,HY,108268529,2020-05-19 11:22:22,|msg=NL_Mai_2|subject=Online+Premiere+%22Kabal...,0,unknown
3,100055559,NO,108268529,2020-05-19 20:46:32,|msg=NL_Mai_2|userAgent=Apple Mail (OS X / Per...,0,Apple Mail (OS X / Personal computer)
4,100055559,HY,108552325,2020-06-02 18:13:24,|msg=NL_Corona_1|subject=Sondernewsletter%3A+J...,1,unknown


In [28]:
irrelevante_iids = df_h[df_h["User Agent"].str.contains('Proxy|Apple Mail|Gmail', case=False)]["iid"].unique()
irrelevante_iids

array([100055559, 100073610, 100073862, 100074164, 100074334, 100076440,
       100081532, 100082082, 100085998, 100090142, 100093208, 100093892,
       100096030, 100110624, 100113202, 100115022, 100119358, 100122034,
       100130428, 100132080, 100132768, 100133030, 100139030, 100142018,
       100147350, 100151884, 100152028, 100153641, 100164202, 100164234,
       100169851, 100171937, 100175321, 100175705, 100183114, 100190662,
       100193728, 100195684, 100200038, 100205305, 100211515, 100213298,
       100215224, 100223814, 100224006, 100228942, 100230944, 100234324,
       100235192, 100247657, 100248875, 100254033, 100256403, 100256545,
       100258522, 100262870, 100274332, 100274876, 100278566, 100292914,
       100302334, 100304874, 100319212, 100336776, 100338930, 100343874,
       100344474, 100346743, 100347353, 100348939, 100352609, 100357367,
       100360041, 100405724, 100406788, 100412492, 100412709, 100416898,
       100419841, 100421193, 100422305, 100424543, 

In [29]:
relevant_iids = df_relevante_iids["iid"].copy()

filtered_iids = [iid for iid in relevant_iids if iid not in irrelevante_iids]

In [30]:
df_filtered_iids = pd.DataFrame(filtered_iids, columns = ["iid"])
df_filtered_iids.head()

,iid
0,157982720
1,100425731
2,107233288
3,137986060
4,158056463


In [14]:
path = x
filename = 'privatkunden.iid.Thalia.ohneAppleEtc.csv'
df_filtered_iids.to_csv(os.path.join(path, filename), index=False)

# Erweiterte Analysen zum Verständnis

##### Ergänzung duch Infos für welche Datenpunkte Probleme bei der Genauigkeit der Öffnungsrate besteht

In [8]:
df_user_agent["User Agent"].fillna("unknown", inplace=True)
df_user_agent_aufgeteilt = df_user_agent

# Spalten mit Ja/Nein-Werten erstellen
df_user_agent_aufgeteilt["unknown"] = df_user_agent['User Agent'].str.contains('unknown', case=False)
#df_user_agent_aufgeteilt['iOS-Nutzer'] = df_user_agent['User Agent'].str.contains('iOS', case=False)
#df_user_agent_aufgeteilt['OS-X-Nutzer'] = df_user_agent['User Agent'].str.contains('OS X|Macintosh', case=False)
df_user_agent_aufgeteilt['Apple-Mail-Nutzer'] = df_user_agent['User Agent'].str.contains('Apple Mail', case=False)
df_user_agent_aufgeteilt['Gmail-Nutzer'] = df_user_agent['User Agent'].str.contains('Gmail', case=False)
df_user_agent_aufgeteilt['Proxy-Server'] = df_user_agent['User Agent'].str.contains('Proxy', case=False)
df_user_agent_aufgeteilt.head()

,User Agent,unknown,Apple-Mail-Nutzer,Gmail-Nutzer,Proxy-Server
0,unknown,True,False,False,False
1,Apple Mail (OS X / Personal computer),False,True,False,False
2,unknown,True,False,False,False
3,Apple Mail (OS X / Personal computer),False,True,False,False
4,unknown,True,False,False,False


In [9]:
for col in df_user_agent.columns[1:]:
    print(df_user_agent[col].value_counts())

True     80292
False    30918
Name: unknown, dtype: int64
False    106187
True       5023
Name: Apple-Mail-Nutzer, dtype: int64
False    108781
True       2429
Name: Gmail-Nutzer, dtype: int64
False    108781
True       2429
Name: Proxy-Server, dtype: int64


##### Tests

In [10]:
df_relevante_iids.info

<bound method DataFrame.info of             iid
0     157982720
1     100425731
2     107233288
3     137986060
4     107233293
...         ...
1757  158031844
1758  100319212
1759  101253103
1760  100466672
1761  100687856

[1762 rows x 1 columns]>